## Nasa Numeric

In [92]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff

#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm 

from sklearn import model_selection as ms

In [93]:
#Carregando dados

data, meta = arff.loadarff('datasets/nasa_numeric.arff')
df = pd.DataFrame(data)
df.head()

,recordnumber,projectname,cat2,forg,center,year,mode,rely,data,cplx,...,acap,aexp,pcap,vexp,lexp,modp,tool,sced,equivphyskloc,act_effort
0,1.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',25.9,117.6
1,2.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',24.6,117.6
2,3.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',7.7,31.2
3,4.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',8.2,36.0
4,5.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',9.7,25.2


## Preparando Dados

In [94]:
def one_hot_enconder(X_transform, transform_data): #X_transform = data, transform_data = colunas
    
    for col in transform_data:
        cols = X_transform[col].unique() #retorna os valores da categoria que serão transformados em colunas 
        col_position = X_transform.columns.get_loc(col) #retorna posição da coluna, a partir dessa posição será adicionado as demais
        col_values = X_transform.pop(col) #retorna os valores da coluna primordial e apaga e remove 
        for c in cols: 
            X_transform.insert(col_position, str(col) + "_" + str(c).replace('b\'','').replace('\'',''), col_values.eq(c))
            col_position += 1

    return X_transform.replace({True: 1, False: 0})          

In [95]:
#removendo atributos desnecessários "recordnumber", "projectname", "year".

df = df.drop(['recordnumber','projectname', 'year'], axis=1)

#removendo linhas com atributos nulos ou vazios

df = df.dropna()

#Convertendo atributos textuais ordinais em numeric com escala de atributos (ainda sem normalização) "OrdinalEncoder"

tranform_data = {b"vl": 0,b"l":1,b"n":2,b"h":3,b"vh":4,b"xh":5}
df = df.replace(tranform_data)

#Convertendo atributos categóricos em colunas. "OneHotEncoder"

transform_data = ['cat2', 'forg', 'center', 'mode']
df =  one_hot_enconder(df, transform_data)

#normalizando dados - #min-max df=(df-df.min())/(df.max()-df.min()) - #zscore df=(df-df.mean())/df.std()

df = (df-df.mean())/df.std()

#separando atributo classe ACT_Effort para entrada nos modelos

X = df.drop(df.columns[-1], axis=1)
y = df.take([-1], axis=1)

## Treinando e avaliando o desempenho dos modelos

In [96]:
models = []
#Instanciando os modelos
models.append(['KNN', KNeighborsRegressor(n_neighbors=5)]) 
models.append(['MLP', MLPRegressor(activation='logistic', solver='sgd', alpha=0.02, max_iter=300, hidden_layer_sizes=200)])
models.append(['RFR', RandomForestRegressor()])
models.append(['SVR', svm.SVR()])

In [97]:
for model in models:
    cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)
    scores = ms.cross_val_score(model[1], X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    #converter para positivo.
    scores = np.absolute(scores)
    s_mean = np.mean(scores)
    s_std = np.std(scores)
    
    print('-', model[0],'------------------------------------')
    print('Scores:', scores, 'MAE: %.4f' % (s_mean), 'STD: %.4f' % (s_std)) 

- KNN ------------------------------------
Scores: [0.49370857 0.61323567 0.3833614 ] MAE: 0.4968 STD: 0.0939
- MLP ------------------------------------
Scores: [0.35208313 0.52384649 0.40527829] MAE: 0.4271 STD: 0.0718
- RFR ------------------------------------
Scores: [0.35534542 0.36679245 0.23614515] MAE: 0.3194 STD: 0.0591
- SVR ------------------------------------
Scores: [0.27411045 0.45665542 0.38445873] MAE: 0.3717 STD: 0.0751


/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/carlos/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached an